In [ ]:
import os
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

ENV_PATH = ".env"
if os.getenv("OPENAI_API_KEY") is None:
    # fallback to reading env file
    from dotenv import load_dotenv

    if not load_dotenv(override=True, dotenv_path=ENV_PATH):
        print(f"failed to load {ENV_PATH}")
        exit(1)
    print(f"loaded {ENV_PATH}")

In [ ]:
prompt = ChatPromptTemplate.from_template("make a haiku about the topic {topic}")
model = ChatOpenAI(model="gpt-3.5-turbo")
output_parser = StrOutputParser()

# the pipe syntax is unique to langchain and is similar to the unix shell pipe
chain = prompt | model | output_parser

chain.invoke({"topic": "sun in the eyes in the car"})

### RAG

In [ ]:
from langchain_community.vectorstores import DocArrayInMemorySearch
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableParallel, RunnablePassthrough
from langchain_openai.chat_models import ChatOpenAI
from langchain_openai.embeddings import OpenAIEmbeddings

vectorstore = DocArrayInMemorySearch.from_texts(
    ["harrison worked at kensho", "bears like to eat honey"],
    embedding=OpenAIEmbeddings(),
)
retriever = vectorstore.as_retriever()
template = """Answer the question based only on the following context:
{context}

Question: {question}
"""

prompt = ChatPromptTemplate.from_template(template)
model = ChatOpenAI(model="gpt-3.5-turbo")
output_parser = StrOutputParser()

setup_and_retrieval = RunnableParallel(
    {"context": retriever, "question": RunnablePassthrough()}
)
chain = setup_and_retrieval | prompt | model | output_parser

# NOTE: you can also manually call the retriever
#retriever.invoke("where did harrison work?")

chain.invoke("where did harrison work?")